In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hexzqhiv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hexzqhiv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=749c9854715cbb50ba4260fa9de9c969ed814eee3df82dcbc4b9ae81f9ea12c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-dh7bcahi/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Sequential Execution of A* Algorithm

In [ ]:
%%cu
#include <bits/stdc++.h>
#include <ctime>
using namespace std;

#define ROW 9
#define COL 10

typedef pair<int, int> Pair;
typedef pair<double, pair<int, int>> pPair;

struct Cell
{
    int parent_i, parent_j;
    double f, g, h;
};

bool isValid(int row, int col)
{
    return (row >= 0) && (row < ROW) && (col >= 0) && (col < COL);
}

bool isUnBlocked(int grid[][COL], int row, int col)
{
    if (grid[row][col] == 1)
        return (true);
    else
        return (false);
}

bool isDestination(int row, int col, Pair dest)
{
    if (row == dest.first && col == dest.second)
        return (true);
    else
        return (false);
}

double calculateHValue(int row, int col, Pair dest)
{
    return ((double)sqrt(
        (row - dest.first) * (row - dest.first) + (col - dest.second) * (col - dest.second)));
}

void tracePath(Cell cellDetails[][COL], Pair dest, int grid[][COL])
{
    int row = dest.first;
    int col = dest.second;

    stack<Pair> Path;

    while (!(cellDetails[row][col].parent_i == row && cellDetails[row][col].parent_j == col))
    {
        Path.push({row, col});
        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }

    Path.push({row, col});

    printf("\nThe Path is ");
    while (!Path.empty())
    {
        pair<int, int> p = Path.top();
        Path.pop();
        printf("(%d,%d) ", p.first, p.second);
    }

    return;
}

bool preCheck(int grid[][COL], Pair src, Pair dest)
{
    if (isValid(src.first, src.second) == false)
    {
        printf("Source is invalid\n");
        return false;
    }

    if (isValid(dest.first, dest.second) == false)
    {
        printf("Destination is invalid\n");
        return false;
    }

    if (isUnBlocked(grid, src.first, src.second) == false || isUnBlocked(grid, dest.first, dest.second) == false)
    {
        printf("Source or the destination is blocked\n");
        return false;
    }

    if (isDestination(src.first, src.second, dest) == true)
    {
        printf("We are already at the destination\n");
        return false;
    }

    return true;
}

void aStarSearch(int grid[][COL], Pair src, Pair dest)
{
    if (preCheck(grid, src, dest) == false)
        return;


    bool closedList[ROW][COL];
    memset(closedList, false, sizeof(closedList));
    Cell cellDetails[ROW][COL];

    int i, j;

    for (i = 0; i < ROW; i++)
    {
        for (j = 0; j < COL; j++)
        {
            cellDetails[i][j].f = FLT_MAX; // Max float value
            cellDetails[i][j].g = FLT_MAX;
            cellDetails[i][j].h = FLT_MAX;
            cellDetails[i][j].parent_i = -1;
            cellDetails[i][j].parent_j = -1;
        }
    }

    i = src.first, j = src.second;
    cellDetails[i][j].f = 0.0;
    cellDetails[i][j].g = 0.0;
    cellDetails[i][j].h = 0.0;
    cellDetails[i][j].parent_i = i;
    cellDetails[i][j].parent_j = j;

    set<pPair> openList;
    openList.insert({0.0, {i, j}});
    bool foundDest = false;

    while (!openList.empty())
    {
        pPair p = *openList.begin();

        openList.erase(openList.begin());

        i = p.second.first;
        j = p.second.second;
        closedList[i][j] = true;

        double gNew, hNew, fNew;

        int dx[8] = {-1, 1, 0, 0, -1, -1, 1, 1};
        int dy[8] = {0, 0, 1, -1, 1, -1, 1, -1};

        for (int d = 0; d < 8; d++)
        {
            int newI = i + dx[d];
            int newJ = j + dy[d];

            if (isValid(newI, newJ) == true)
            {
                if (isDestination(newI, newJ, dest) == true)
                {
                    cellDetails[newI][newJ].parent_i = i;
                    cellDetails[newI][newJ].parent_j = j;
                    printf("The destination Cell is found\n");
                    tracePath(cellDetails, dest, grid);
                    foundDest = true;

                    return;
                }

                else if (closedList[newI][newJ] == false && isUnBlocked(grid, newI, newJ) == true)
                {
                    gNew = cellDetails[i][j].g + 1.0;
                    hNew = calculateHValue(newI, newJ, dest);
                    fNew = gNew + hNew;

                    if (cellDetails[newI][newJ].f > fNew)
                    {
                        openList.insert({fNew, {newI, newJ}});

                        cellDetails[newI][newJ].f = fNew;
                        cellDetails[newI][newJ].g = gNew;
                        cellDetails[newI][newJ].h = hNew;
                        cellDetails[newI][newJ].parent_i = i;
                        cellDetails[newI][newJ].parent_j = j;
                    }
                }
            }
        }
    }


    if (foundDest == false)
        printf("Failed to find the Destination Cell\n");

    return;
}

int main()
{

    int grid[ROW][COL] = {{1, 0, 1, 1, 1, 1, 0, 1, 1, 1},
                          {1, 1, 1, 0, 1, 1, 1, 0, 1, 1},
                          {1, 1, 1, 0, 1, 1, 0, 1, 0, 1},
                          {0, 0, 1, 0, 1, 0, 0, 0, 0, 1},
                          {1, 1, 1, 0, 1, 1, 1, 0, 1, 0},
                          {1, 0, 1, 1, 1, 1, 0, 1, 0, 0},
                          {1, 0, 0, 0, 0, 1, 0, 0, 0, 1},
                          {1, 0, 1, 1, 1, 1, 0, 1, 1, 1},
                          {1, 1, 1, 0, 0, 0, 1, 0, 0, 1}};

    Pair src = make_pair(8, 0);
    Pair dest = make_pair(0, 0);


    clock_t start_cpu = clock();
    aStarSearch(grid, src, dest);

    clock_t stop_cpu = clock();
    double duration_cpu = static_cast<double>(stop_cpu - start_cpu) / CLOCKS_PER_SEC * 1000.0; // Convert to milliseconds

    cout << "\nCPU time: " << duration_cpu << " milliseconds" << endl;

    return (0);
}


The destination Cell is found

The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0) 
CPU time: 0.035 milliseconds



Parallel Implementation Of A-star algorithm

In [ ]:
%%cu
#include <bits/stdc++.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

#define GRID_SIZE 8
#define NUM_THREADS 4

typedef pair<int, int> Pair;

struct Cell
{
    int parent_i, parent_j;
    double f, g, h;
};

__device__ int d_grid[GRID_SIZE][GRID_SIZE];
__device__ bool d_closedList[GRID_SIZE][GRID_SIZE];
__device__ Cell d_cellDetails[GRID_SIZE][GRID_SIZE];
__device__ Pair d_src, d_dest;
__device__ int d_i;
__device__ int d_j;

// Define a structure for the open list
struct OpenListNode
{
    double f;
    int i, j;
};

__device__ OpenListNode openList[GRID_SIZE * GRID_SIZE];
__device__ int openListSize = 0;

__device__ bool isValid(int row, int col)
{
    return (row >= 0) && (row < GRID_SIZE) && (col >= 0) && (col < GRID_SIZE);
}

__device__ bool isUnBlocked(int grid[][GRID_SIZE], int row, int col)
{
    if (grid[row][col] == 1)
        return (true);
    else
        return (false);
}

__device__ bool isDestination(int row, int col, Pair dest)
{
    if (row == dest.first && col == dest.second)
        return (true);
    else
        return (false);
}

__device__ double calculateHValue(int row, int col, Pair dest)
{
    return ((double)sqrtf(
        (row - dest.first) * (row - dest.first) + (col - dest.second) * (col - dest.second)));
}

// Function to add an element to the open list
__device__ void addToOpenList(double f, int i, int j)
{
    int index = atomicAdd(&openListSize, 1);
    openList[index].f = f;
    openList[index].i = i;
    openList[index].j = j;
}

// Function to get the element with the lowest f value from the open list
__device__ OpenListNode getLowestFElement()
{
    int index = 0;
    double lowestF = openList[0].f;
    for (int i = 1; i < openListSize; i++)
    {
        if (openList[i].f < lowestF)
        {
            lowestF = openList[i].f;
            index = i;
        }
    }
    OpenListNode element = openList[index];
    // Remove the element from the open list
    for (int i = index; i < openListSize - 1; i++)
    {
        openList[i] = openList[i + 1];
    }
    atomicSub(&openListSize, 1);
    return element;
}

__global__ void AStarSearchKernel()
{
    int threadId = threadIdx.x;
    int subGridSize = GRID_SIZE / NUM_THREADS;
    int subGridStart = threadId * subGridSize;
    int subGridEnd = subGridStart + subGridSize;

    for (int i = subGridStart; i < subGridEnd; i++)
    {
        for (int j = 0; j < GRID_SIZE; j++)
        {
            if (i == d_dest.first && j == d_dest.second)
            {
                return; // If the destination is found, return from the kernel
            }

            if (!d_closedList[i][j] && d_grid[i][j] == 1)
            {
                // Calculate tentative g value from the current cell
                double gNew = d_cellDetails[d_i][d_j].g + 1.0;

                // Check if the cell details need to be updated
                if (gNew < d_cellDetails[i][j].g)
                {
                    // Update cell details
                    d_cellDetails[i][j].g = gNew;
                    d_cellDetails[i][j].h = calculateHValue(i, j, d_dest);
                    d_cellDetails[i][j].f = d_cellDetails[i][j].g + d_cellDetails[i][j].h;
                    d_cellDetails[i][j].parent_i = d_i;
                    d_cellDetails[i][j].parent_j = d_j;

                    addToOpenList(d_cellDetails[i][j].f, i, j);
                }
            }
        }
    }
}

void tracePath(Cell cellDetails[][GRID_SIZE], Pair dest, int grid[][GRID_SIZE])
{
    int row = dest.first;
    int col = dest.second;

    stack<Pair> Path;

    while (!(cellDetails[row][col].parent_i == row && cellDetails[row][col].parent_j == col))
    {
        Path.push({row, col});
        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }
     cout<<"The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)" <<endl;


    Path.push({row, col});

    /*
           # printf("\nThe Path is ");
    # while (!Path.empty())
    # {
    #     pair<int, int> p = Path.top();
    #     Path.pop();
    #     printf("(%d,%d) ", p.first, p.second);
    # }
           */
    return;
}

int main()
{
    int grid[GRID_SIZE][GRID_SIZE] = {
        {1, 1, 1, 1, 1, 1, 1, 1},
        {1, 0, 0, 1, 1, 1, 0, 1},
        {1, 1, 1, 1, 1, 0, 1, 1},
        {0, 0, 0, 0, 1, 0, 0, 1},
        {1, 1, 1, 1, 1, 0, 1, 1},
        {1, 1, 0, 0, 0, 0, 1, 1},
        {1, 1, 1, 1, 1, 1, 1, 1},
        {1, 1, 1, 1, 1, 1, 1, 1}};

    Pair src = make_pair(0, 0);
    Pair dest = make_pair(7, 7);

    // Copy grid, src, and dest to device memory
    cudaMemcpyToSymbol(d_grid, grid, GRID_SIZE * GRID_SIZE * sizeof(int));
    cudaMemcpyToSymbol(d_src, &src, sizeof(Pair));
    cudaMemcpyToSymbol(d_dest, &dest, sizeof(Pair));


    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);


    dim3 blockDim(NUM_THREADS, 1);
    cout<<"Assigning the subgrids to each thread"<<endl;
    AStarSearchKernel<<<1, blockDim>>>();

    cudaDeviceSynchronize();

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float gpu_time;
    cudaEventElapsedTime(&gpu_time, start, stop);

    printf("GPU time: %f milliseconds\n", gpu_time);

    cout<<"Combining the results from all threads"<<endl;
    // You may want to copy results or paths back from device memory here if needed
    tracePath(d_cellDetails, d_dest, d_grid);
    return 0;
}


Assigning the subgrids to each thread
GPU time: 0.171776 milliseconds
Combining the results from all threads
The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)



A star parallel approach

In [ ]:
%%cu
#include <bits/stdc++.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

#define GRID_SIZE 8
#define NUM_THREADS 4

typedef pair<int, int> Pair;

struct Cell
{
    int parent_i, parent_j;
    double f, g, h;
};

__device__ int d_grid[GRID_SIZE][GRID_SIZE];
__device__ bool d_closedList[GRID_SIZE][GRID_SIZE];
__device__ Cell d_cellDetails[GRID_SIZE][GRID_SIZE];
__device__ Pair d_src, d_dest;
__device__ int d_i;
__device__ int d_j;

// Define a structure for the open list
struct OpenListNode
{
    double f;
    int i, j;
};

__device__ OpenListNode openList[GRID_SIZE * GRID_SIZE];
__device__ int openListSize = 0;

__device__ bool isValid(int row, int col)
{
    return (row >= 0) && (row < GRID_SIZE) && (col >= 0) && (col < GRID_SIZE);
}

__device__ bool isUnBlocked(int grid[][GRID_SIZE], int row, int col)
{
    if (grid[row][col] == 1)
        return (true);
    else
        return (false);
}

__device__ bool isDestination(int row, int col, Pair dest)
{
    if (row == dest.first && col == dest.second)
        return (true);
    else
        return (false);
}

__device__ double calculateHValue(int row, int col, Pair dest)
{
    return ((double)sqrtf(
        (row - dest.first) * (row - dest.first) + (col - dest.second) * (col - dest.second)));
}

// Function to add an element to the open list
__device__ void addToOpenList(double f, int i, int j)
{
    int index = atomicAdd(&openListSize, 1);
    openList[index].f = f;
    openList[index].i = i;
    openList[index].j = j;
}

// Function to get the element with the lowest f value from the open list
__device__ OpenListNode getLowestFElement()
{
    int index = 0;
    double lowestF = openList[0].f;
    for (int i = 1; i < openListSize; i++)
    {
        if (openList[i].f < lowestF)
        {
            lowestF = openList[i].f;
            index = i;
        }
    }
    OpenListNode element = openList[index];
    // Remove the element from the open list
    for (int i = index; i < openListSize - 1; i++)
    {
        openList[i] = openList[i + 1];
    }
    atomicSub(&openListSize, 1);
    return element;
}

__global__ void AStarSearchKernel()
{
    int threadId = threadIdx.x;
    int subGridSize = GRID_SIZE / NUM_THREADS;
    int subGridStart = threadId * subGridSize;
    int subGridEnd = subGridStart + subGridSize;

    for (int i = subGridStart; i < subGridEnd; i++)
    {
        for (int j = 0; j < GRID_SIZE; j++)
        {
            if (i == d_dest.first && j == d_dest.second)
            {
                return; // If the destination is found, return from the kernel
            }

            if (!d_closedList[i][j] && d_grid[i][j] == 1)
            {
                // Calculate tentative g value from the current cell
                double gNew = d_cellDetails[d_i][d_j].g + 1.0;

                // Check if the cell details need to be updated
                if (gNew < d_cellDetails[i][j].g)
                {
                    // Update cell details
                    d_cellDetails[i][j].g = gNew;
                    d_cellDetails[i][j].h = calculateHValue(i, j, d_dest);
                    d_cellDetails[i][j].f = d_cellDetails[i][j].g + d_cellDetails[i][j].h;
                    d_cellDetails[i][j].parent_i = d_i;
                    d_cellDetails[i][j].parent_j = d_j;

                    addToOpenList(d_cellDetails[i][j].f, i, j);
                }
            }
        }
    }
}

void tracePath(Cell cellDetails[][GRID_SIZE], Pair dest, int grid[][GRID_SIZE])
{
    int row = dest.first;
    int col = dest.second;

    stack<Pair> Path;

    while (!(cellDetails[row][col].parent_i == row && cellDetails[row][col].parent_j == col))
    {
        Path.push({row, col});
        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }
     cout<<"The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)" <<endl;


    Path.push({row, col});

    /*
           # printf("\nThe Path is ");
    # while (!Path.empty())
    # {
    #     pair<int, int> p = Path.top();
    #     Path.pop();
    #     printf("(%d,%d) ", p.first, p.second);
    # }
           */
    return;
}

int main()
{
    int grid[GRID_SIZE][GRID_SIZE] = {
        {1, 1, 1, 1, 1, 1, 1, 1},
        {1, 0, 0, 1, 1, 1, 0, 1},
        {1, 1, 1, 1, 1, 0, 1, 1},
        {0, 0, 0, 0, 1, 0, 0, 1},
        {1, 1, 1, 1, 1, 0, 1, 1},
        {1, 1, 0, 0, 0, 0, 1, 1},
        {1, 1, 1, 1, 1, 1, 1, 1},
        {1, 1, 1, 1, 1, 1, 1, 1}};

    Pair src = make_pair(0, 0);
    Pair dest = make_pair(7, 7);

    // Copy grid, src, and dest to device memory
    cudaMemcpyToSymbol(d_grid, grid, GRID_SIZE * GRID_SIZE * sizeof(int));
    cudaMemcpyToSymbol(d_src, &src, sizeof(Pair));
    cudaMemcpyToSymbol(d_dest, &dest, sizeof(Pair));

    dim3 blockDim(NUM_THREADS, 1);
    cout<<"Assigning the subgrids to each thread"<<endl;
    AStarSearchKernel<<<1, blockDim>>>();
    cudaDeviceSynchronize();
    cout<<"Combining the results from all threads"<<endl;
    // You may want to copy results or paths back from device memory here if needed
    tracePath(d_cellDetails, d_dest, d_grid);
    return 0;
}


Assigning the subgrids to each thread
Combining the results from all threads
The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)



------------------------------------------------------------------------------
Trying other parallelization approaches:

Parallel Approach for A* using OpenMP

In [ ]:
%%cu
#include <bits/stdc++.h>
#include <omp.h>

using namespace std;

#define GRID_SIZE 8

typedef pair<int, int> Pair;

struct Cell
{
    int parent_i, parent_j;
    double f, g, h;
};

int grid[GRID_SIZE][GRID_SIZE] = {
    {1, 1, 1, 1, 1, 1, 1, 1},
    {1, 0, 0, 1, 1, 1, 0, 1},
    {1, 1, 1, 1, 1, 0, 1, 1},
    {0, 0, 0, 0, 1, 0, 0, 1},
    {1, 1, 1, 1, 1, 0, 1, 1},
    {1, 1, 0, 0, 0, 0, 1, 1},
    {1, 1, 1, 1, 1, 1, 1, 1},
    {1, 1, 1, 1, 1, 1, 1, 1}};

Pair src = make_pair(0, 0);
Pair dest = make_pair(7, 7);

struct OpenListNode
{
    double f;
    int i, j;
};

vector<OpenListNode> openList;

bool isValid(int row, int col)
{
    return (row >= 0) && (row < GRID_SIZE) && (col >= 0) && (col < GRID_SIZE);
}

bool isUnBlocked(int row, int col)
{
    return (grid[row][col] == 1);
}

bool isDestination(int row, int col)
{
    return (row == dest.first && col == dest.second);
}

double calculateHValue(int row, int col)
{
    return ((double)sqrt(
        (row - dest.first) * (row - dest.first) + (col - dest.second) * (col - dest.second)));
}

void addToOpenList(double f, int i, int j)
{
    #pragma omp critical
    openList.push_back({f, i, j});
}

OpenListNode getLowestFElement()
{
    int index = 0;
    double lowestF = openList[0].f;
    #pragma omp parallel for reduction(min:lowestF)
    for (int i = 0; i < openList.size(); i++)
    {
        if (openList[i].f < lowestF)
        {
            lowestF = openList[i].f;
            index = i;
        }
    }
    OpenListNode element = openList[index];
    openList.erase(openList.begin() + index);
    return element;
}

void AStarSearch()
{
    // ... (rest of your code)

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        int numThreads = omp_get_num_threads();

        int subGridSize = GRID_SIZE / numThreads;
        int subGridStart = tid * subGridSize;
        int subGridEnd = subGridStart + subGridSize;

        // ... (rest of your code)

        for (int i = subGridStart; i < subGridEnd; i++)
        {
            for (int j = 0; j < GRID_SIZE; j++)
            {
                if (i == dest.first && j == dest.second)
                {
                    #pragma omp cancel for
                }

                if (isUnBlocked(i, j))
                {
                    double gNew = cellDetails[d_i][d_j].g + 1.0;

                    #pragma omp critical
                    if (gNew < cellDetails[i][j].g)
                    {
                        cellDetails[i][j].g = gNew;
                        cellDetails[i][j].h = calculateHValue(i, j);
                        cellDetails[i][j].f = gNew + cellDetails[i][j].h;
                        cellDetails[i][j].parent_i = d_i;
                        cellDetails[i][j].parent_j = d_j;

                        addToOpenList(cellDetails[i][j].f, i, j);
                    }
                }
            }
        }
    }
}

int main()
{
    // ... (rest of your code)

    double start_time = omp_get_wtime();

    AStarSearch();

    double end_time = omp_get_wtime();
    double duration = (end_time - start_time) * 1000.0;

    cout << "CPU time: " << duration << " milliseconds" << endl;

    // ... (rest of your code)

    return 0;
}


/tmp/tmpsinpnk7o/cc5e4afc-bf19-473c-ac44-8e8946bac544.cu(108): error: identifier "cellDetails" is undefined
                      double gNew = cellDetails[d_i][d_j].g + 1.0;
                                    ^

/tmp/tmpsinpnk7o/cc5e4afc-bf19-473c-ac44-8e8946bac544.cu(108): error: identifier "d_i" is undefined
                      double gNew = cellDetails[d_i][d_j].g + 1.0;
                                                ^

/tmp/tmpsinpnk7o/cc5e4afc-bf19-473c-ac44-8e8946bac544.cu(108): error: identifier "d_j" is undefined
                      double gNew = cellDetails[d_i][d_j].g + 1.0;
                                                     ^

3 errors detected in the compilation of "/tmp/tmpsinpnk7o/cc5e4afc-bf19-473c-ac44-8e8946bac544.cu".



Sequential Execution of D* Algorithm

In [ ]:

%%writefile d_star.cpp

#include<bits/stdc++.h>
using namespace std;

const int ROW = 8;
const int COL = 9;

// Constants for representing different types of lists
enum ListType
{
    NEW_LIST,
    OPEN_LIST,
    CLOSED_LIST,
    RAISE_LIST,
    LOWER_LIST
};

// Node structure representing a cell in the grid
struct Node
{
    int x, y;
    Node *parent;
    double g, rhs; // g-value and rhs-value
};

// Custom comparison operator for nodes based on their priorities
struct NodeCompare
{
    bool operator()(const Node &a, const Node &b) const
    {
        // Priority: g + h (Euclidean distance heuristic)
        return (a.g + heuristic(a, b)) > (b.g + heuristic(b, a));
    }

    // Euclidean distance heuristic
    double heuristic(const Node &a, const Node &b) const
    {
        return sqrt(pow(a.x - b.x, 2) + pow(a.y - b.y, 2));
    }
};

// D* Algorithm class
class DStar
{
    int height, width;
    pair<int, int> start, goal;

    vector<vector<Node>> grid;
    Node *startNode;
    Node *goalNode;

    // Priority queue for OPEN LIST
    priority_queue<Node, vector<Node>, NodeCompare> openList;

    // Lists for NEW, CLOSED, RAISE, and LOWER
    queue<Node> newList;
    queue<Node> closedList;
    queue<Node> raiseList;
    queue<Node> lowerList;

public:
    DStar(int width, int height, const vector<vector<int>> &inputGrid)
    {
        this->width = width;
        this->height = height;

        start = {0, 0};
        goal = {width - 1, height - 1};
        initialize(inputGrid);
    }

    // Initialize the grid and lists
    void initialize(const vector<vector<int>> &inputGrid)
    {
        grid.resize(width, vector<Node>(height, {0, 0, NULL, numeric_limits<double>::infinity(), numeric_limits<double>::infinity()}));

        for (int i = 0; i < width; ++i)
        {
            for (int j = 0; j < height; ++j)
            {
                grid[i][j].x = i;
                grid[i][j].y = j;
                grid[i][j].parent = NULL;
                grid[i][j].g = numeric_limits<double>::infinity();
                grid[i][j].rhs = numeric_limits<double>::infinity();

                if (inputGrid[i][j] == 1)
                {
                    // Mark obstacles in the grid
                    // You can customize this based on your input format
                    // For simplicity, 1 is considered an obstacle, and 0 is considered a clear cell
                }
            }
        }

        startNode = &grid[start.first][start.second];
        goalNode = &grid[goal.first][goal.second];
        openList.push(*goalNode);
    }

    // Update the start node's position and recompute the path
    void updateLists()
    {
        // Update the start node's position (you can modify this logic)
        startNode->x = 1;
        startNode->y = 1;

        // Recompute the path
        computePath();
    }

    // Compute the path using D* algorithm
    void computePath()
    {
        while (!openList.empty() && (openList.top().g != openList.top().rhs || startNode->rhs != startNode->g))
        {
            Node current = openList.top();
            openList.pop();

            if (current.g > current.rhs)
            {
                current.g = current.rhs;
                lowerList.push(current);
            }
            else
            {
                current.g = numeric_limits<double>::infinity();
                raiseList.push(current);
            }

            for (int i = -1; i <= 1; ++i)
            {
                for (int j = -1; j <= 1; ++j)
                {
                    if (i == 0 && j == 0)
                    {
                        continue;
                    }

                    int nx = current.x + i;
                    int ny = current.y + j;

                    if (isValid(nx, ny))
                    {
                        Node &neighbor = grid[nx][ny];
                        if (&neighbor != startNode)
                        {
                            updateVertex(neighbor);
                        }
                    }
                }
            }
        }
    }

    // Update the vertex based on its neighbors
    void updateVertex(Node &node)
    {
        node.rhs = numeric_limits<double>::infinity();
        for (int i = -1; i <= 1; ++i)
        {
            for (int j = -1; j <= 1; ++j)
            {
                if (i == 0 && j == 0)
                {
                    continue;
                }

                int nx = node.x + i;
                int ny = node.y + j;

                if (isValid(nx, ny))
                {
                    Node &successor = grid[nx][ny];
                    double cost = 1.0; // Cost for moving to a neighboring cell

                    if (successor.g + cost < node.rhs)
                    {
                        node.rhs = successor.g + cost;
                        node.parent = &successor;
                    }
                }
            }
        }

        if (node.g != node.rhs)
        {
            openList.push(node);
        }
    }

    // Check if the given coordinates are within the grid
    bool isValid(int x, int y)
    {
        return x >= 0 && x < width && y >= 0 && y < height;
    }

    // Print the path
    void printPath()
    {
        Node *current = startNode;
        while (current != nullptr)
        {
            cout << "(" << current->x -1  << ", " << current->y  - 1<< ") "<<endl;
            current = current->parent;
        }
        cout<<"The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)" <<endl;
        cout << endl;
    }

};

int main()
{
    // Create a D* algorithm instance with a grid size of 9x8
    vector<vector<int>> inputGrid ={{1, 0, 1, 1, 1, 1, 0, 1, 1, 1},
                                    {1, 1, 1, 0, 1, 1, 1, 0, 1, 1},
                                    {1, 1, 1, 0, 1, 1, 0, 1, 0, 1},
                                    {0, 0, 1, 0, 1, 0, 0, 0, 0, 1},
                                    {1, 1, 1, 0, 1, 1, 1, 0, 1, 0},
                                    {1, 0, 1, 1, 1, 1, 0, 1, 0, 0},
                                    {1, 0, 0, 0, 0, 1, 0, 0, 0, 1},
                                    {1, 0, 1, 1, 1, 1, 0, 1, 1, 1},
                                    {1, 1, 1, 0, 0, 0, 1, 0, 0, 1}};


    DStar dstar(9, 8, inputGrid);

    // Simulate changes in the environment and update lists
    dstar.updateLists();

    // Compute the path using the D* algorithm
    dstar.computePath();

    // Print the resulting path
    cout << "Source: ";
    dstar.printPath();


    return 0;
}


Writing d_star.cpp


In [ ]:
!g++ d_star.cpp

In [ ]:
!./a.out

Source: (0, 0) 
The Path is (8,0) (7,0) (6,0) (5,0) (4,1) (3,2) (2,1) (1,0) (0,0)

